In [159]:
import numpy as np
import pandas as pd

In [161]:
from sklearn.impute import SimpleImputer  #i imported this because i have null values in my column
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [163]:
df = pd.read_csv('covid_toy.csv')

In [165]:
df.head() #for gender and city apply nominal categorical encoding and for cough apply ordinal encoding

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [167]:
df['cough'].value_counts() #not count its 'counts'

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [169]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [171]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [173]:
X_train

,age,gender,fever,cough,city
84,69,Female,98.0,Strong,Mumbai
10,75,Female,NaN,Mild,Delhi
20,12,Male,98.0,Strong,Bangalore
40,49,Female,102.0,Mild,Delhi
72,83,Female,101.0,Mild,Kolkata
...,...,...,...,...,...
9,64,Female,101.0,Mild,Delhi
23,80,Female,98.0,Mild,Delhi
78,11,Male,100.0,Mild,Bangalore
43,22,Female,99.0,Mild,Bangalore


In [175]:
X_test.shape

(20, 5)

In [177]:
y_train

84     No
10     No
20     No
40     No
72     No
     ... 
9      No
23    Yes
78    Yes
43    Yes
0      No
Name: has_covid, Length: 80, dtype: object

In [179]:
y_test

90     No
7     Yes
6      No
41    Yes
35     No
97     No
91    Yes
44     No
51    Yes
56     No
96    Yes
57     No
69     No
65     No
71     No
31     No
49     No
2      No
14     No
3      No
Name: has_covid, dtype: object

## 1. Aam Zindagi

In [182]:
# adding simple imputer to fever col  -- this is for filling null values in the fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
''' in this we apply different encoding on different columns and combine them'''
# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever.shape #you can see the values by removing the shape 

(80, 1)

In [184]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['mild','strong']])  #larger value for mild and smaller value for strong
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

ValueError: Found unknown categories ['Strong', 'Mild'] in column 0 during fit

In [157]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False) #droping out the first column
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [142]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values #here we are taking only age column and taking out all the values and storing in some variable for both train and test

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [144]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1) #now concatenating all the columns
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Mentos Zindagi

In [147]:
from sklearn.compose import ColumnTransformer

In [149]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']), #must be in tuple and this tuple includes the 3 values 1 -> transformer name, 2 -> tranformer object, 3 -> list with on which columns u want to apply that transformation
    ('tnf2',OrdinalEncoder(categories=[['mild','strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')  #remainder has 2 values drop or passthrough -> means voo column ku koi ched chad nahi karega

In [151]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ("tnf2", OrdinalEncoder(categories=[['mild', 'strong']]), ['cough']),
    ("tnf3", OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder="passthrough")


In [153]:
transformer.fit_transform(X_train).shape

ValueError: Found unknown categories ['Strong', 'Mild'] in column 0 during fit

In [ ]:
transformer.transform(X_test).shape